In [ ]:
import geopandas as gpd
import os

# === PATHS ===
base_dir = r"C:\Users\natda\OneDrive - Northeastern University\Desktop\NatDave\Academics\PhD_NU\RESEARCH\Traffic_Stress\Boston"
roads_path = os.path.join(base_dir, "street_network.shp")
bikes_path = os.path.join(base_dir, "Bike_Network_Jan_2025.shp")

# === PARAMETERS ===
BIKE_BUF = 5        # Expand bike lanes by a buffer (for calc. only)
OLAP_LEN = 0.15     # Min % of bike lane that must overlap a road

# === LOAD AND PREPARE DATA ===
roads, bikes = gpd.read_file(roads_path), gpd.read_file(bikes_path)
bikes = bikes.to_crs(roads.crs).dropna(subset=["geometry"])
roads = roads.dropna(subset=["geometry"])
bikes["buffered_geom"] = bikes.geometry.buffer(BIKE_BUF)
roads_sindex = roads.sindex

# === FUNCTIONS ===
def check_road_olap(bike_geom):
    """
    Determines if a bike lane has major overlap with roads.
    """
    if bike_geom is None or bike_geom.is_empty: return False
    candidates = roads.iloc[list(roads_sindex.intersection(bike_geom.bounds))]
    if candidates.empty: return False
    shared_len = bike_geom.intersection(candidates.geometry.union_all()).length
    return shared_len / bike_geom.length >= OLAP_LEN

def find_overlapping_road_ids(bike_geom):
    """
    Finds IDs of road segments that overlap with the bike segment's buffered region.
    """
    if bike_geom is None or bike_geom.is_empty: return []
    candidates = roads.iloc[list(roads_sindex.intersection(bike_geom.bounds))]
    overlapping_ids = [
        road["unique_id"]
        for _, road in candidates.iterrows()
        if bike_geom.intersection(road.geometry).length / bike_geom.length >= OLAP_LEN
    ]
    return overlapping_ids


# === DICTIONARY OF BIKE-ROAD OVERLAPS ===
bike_road_overlap_dict = {}

# Iterate through bike segments and populate the dictionary
for _, bike in bikes.iterrows():
    bike_id = bike["unique_id"]
    overlapping_road_ids = find_overlapping_road_ids(bike["buffered_geom"])
    bike_road_overlap_dict[bike_id] = overlapping_road_ids

# === FILTER NON-OVERLAPPING BIKE SEGMENTS ===
bikes["olap_flag"] = bikes["buffered_geom"].apply(check_road_olap)
bikes_no_olap = bikes[~bikes["olap_flag"]].copy()
bikes_no_olap["geometry"] = bikes.loc[bikes_no_olap.index, "geometry"]
bikes_no_olap = bikes_no_olap.drop(columns=["buffered_geom"])

# === SAVE OUTPUT ===
output_path = os.path.join(base_dir, "bikes_no_olap.shp")
bikes_no_olap.to_file(output_path, driver="ESRI Shapefile")
print(f"Saved {len(bikes_no_olap)} bike segments with < {OLAP_LEN * 100}% shared length.")

Saved 133 bike segments with < 15.0% shared length.


In [2]:
# Neponset River Trail
bike_road_overlap_dict[3007]

[17581, 18472]

In [3]:
# University Dr
bike_road_overlap_dict[3381]

[]

In [4]:
# NEU footbridge
bike_road_overlap_dict[3468]

[]

In [5]:
# === INITIALIZE bike_type2 COLUMN IN ROADS DATAFRAME ===
roads["bike_type2"] = None

# === FUNCTION TO ASSIGN bike_type2 BASED ON OVERLAP ===
def assign_bike_type_to_roads():
    """Populates the 'bike_type2' column in the roads DataFrame based on overlap with bike lanes."""
    for bike_id, overlapping_road_ids in bike_road_overlap_dict.items():
        # Find the ExisFacil value for the current bike segment
        bike_exisfacil_value = bikes.loc[bikes["unique_id"] == bike_id, "ExisFacil"].values
        if bike_exisfacil_value.size == 0:
            continue  # Skip if no ExisFacil value is found for the bike segment

        # Assign the ExisFacil value to the corresponding road segments in the 'bike_type2' column
        for road_id in overlapping_road_ids:
            roads.loc[roads["unique_id"] == road_id, "bike_type2"] = bike_exisfacil_value[0]

# === ASSIGN bike_type2 VALUES TO ROADS BASED ON OVERLAP ===
assign_bike_type_to_roads()

# === SAVE THE UPDATED ROADS SHAPEFILE ===
roads.to_file(roads_path, driver="ESRI Shapefile")

In [6]:
roads["bike_type2"].notna().sum()

3418